In [ ]:
import numpy as np
import math
from numpy import linalg
np.set_printoptions(precision=5, suppress=True)

In [ ]:
def AxisAngle2A(pphi):
 
    p = np.array(pphi[:3])
    phi = pphi[3]

    n = np.linalg.norm(p)
    if n > 0:
        p = p/n

    m = p.reshape(3, 1) @ p.reshape(1, 3)

    px = np.array(
        [[   0,  -p[2],  p[1]],
         [ p[2],    0,  -p[0]],
         [-p[1],  p[0],    0]]
    )

    E = np.eye(3)

    s = np.sin(phi)
    c = np.cos(phi)

    A = m + c*(E - m) + s*px

    A = np.where(np.isclose(A, 0) , 0.0 , A)
    
    return A

In [ ]:
def Euler2A(uglovi): 

    psi   = uglovi[0]
    theta = uglovi[1]
    phi   = uglovi[2]

    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(phi), -np.sin(phi)],
        [0, np.sin(phi), np.cos(phi)]
    ])

    Ry = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])

    Rz = np.array([
        [np.cos(psi), -np.sin(psi), 0],
        [np.sin(psi), np.cos(psi), 0],
        [0, 0, 1]
    ])

    A = Rz @ Ry @ Rx

    A = np.where(np.isclose(A, 0) , 0 , A) 

    return A

In [ ]:
def A2AxisAngle(A):

    if not np.allclose(A @ A.T, np.eye(3), atol=1e-6) or not np.isclose(linalg.det(A), 1.0, atol=1e-6):
        return "Nije matrica kretanja!"
    
    cos_phi = (np.trace(A) - 1) / 2.0
    cos_phi = np.clip(cos_phi, -1.0, 1.0)
    phi = np.arccos(cos_phi)

    if np.isclose(phi, 0):
        return np.array([1, 0, 0]), 0
    
    if np.isclose(phi, np.pi):
        values, vectors = linalg.eig(A)
        idx = np.argmin(np.abs(values - 1.0))
        p = np.real(vectors[:, idx])
    else:
        px = A[2, 1] - A[1, 2]
        py = A[0, 2] - A[2, 0]
        pz = A[1, 0] - A[0, 1]
        p = np.array([px, py, pz])

    p = p / linalg.norm(p)

    pphi = np.array([p[0], p[1], p[2], phi])
    pphi = np.where(np.isclose(pphi, 0) , 0 , pphi)

    return pphi

In [ ]:
def A2Euler(A):

    if not np.allclose(A @ A.T, np.eye(3), atol=1e-6) or not np.isclose(linalg.det(A), 1.0, atol=1e-6):
        return "Nije matrica kretanja!"
    
    if np.isclose(np.abs(A[2, 0]), 1):
        phi = 0.0
        theta = -np.sign(A[2, 0])*np.pi / 2
        psi = np.arctan2(-A[0, 1], A[1, 1])
    else:
        phi = np.arctan2(A[2, 1], A[2, 2])
        theta = np.arcsin(-A[2, 0])
        psi = np.arctan2(A[1, 0], A[0, 0])

    uglovi = np.array([psi, theta, phi])
    uglovi = np.where(np.isclose(uglovi, 0) , 0 , uglovi)  

    return uglovi